In [ ]:
#######################################################
#######################################################
#python code for stacking
#######################################################
#######################################################
import os
import sys
import numpy as np
import astropy.io.fits as pyfits
from astropy.coordinates import ICRS
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
from astropy.coordinates import SkyCoord

from scipy import stats
from scipy import optimize #Leastsq Levenberg-Marquadt Algorithm
from scipy.interpolate import UnivariateSpline
from scipy import interpolate
#for plotting
import matplotlib
import matplotlib.pyplot as plt
from scipy.signal import medfilt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widget
from IPython.display import Image
import MontagePy.main as mon

In [ ]:
user=widget.Text(
    value='username',
    description='Enter username (no spaces):',
    layout={'width': '300px'},style = {'description_width': 'initial'}
)
display(user)

In [ ]:
raw_images = 'upload_raw_images_here'
pathout = 'final_stacked'
filters='filters'
username=user.value


In [ ]:
#make list of the files in the archive directory
myfilelist = [ ff for ff in np.sort(os.listdir(raw_images)) if '.fits.fz' in ff ]
print('list of raw images in upload folder:')
myfilelist

In [ ]:
#clean out the to-add directories
os.system('rm -rf %s/*' % filters)
filtlist=[]
#loop over all files and sort them into the correct to-add directories
for f in myfilelist:
        #read in the files
        inhdulist = pyfits.open(raw_images+'/'+f)
        #get the filter name and create directory
        fil = inhdulist[1].header['FILTER']
        objectname = inhdulist[1].header['OBJECT']
        filtlist.append(fil)
        filter_folder=os.path.join(filters,fil)
        if os.path.exists(filter_folder) == False:
            os.mkdir(filter_folder)
            print('creating directory for filter:',fil)
        #save the files in pathout directory
        pyfits.writeto(filter_folder+"/"+f.strip('.fz'), inhdulist[1].data, inhdulist[1].header)

In [ ]:
try:
    os.remove('filters/.DS_Store')#remove temp file
except:
    pass
print('List of filters in raw data:')
print(list(np.unique(filtlist)))
print('for object:',objectname)

In [ ]:
def stack_images(pathlist,fil,pathout,username,objectname):
    #for fil in os.listdir(pathlist):
    print('---stacking images for filter:',fil)
    add_dir = os.path.join(pathlist,fil)
    #make list of the files in the toadd directory
    myfilelist = [ fff for fff in np.sort(os.listdir(add_dir)) if '.fits' in fff ]

    os.system('rm -rf montage_rot')
    os.system('rm -rf montage_diff')
    os.system('rm -rf montage_corr')
    os.system('mkdir montage_rot')
    os.system('mkdir montage_diff')
    os.system('mkdir montage_corr')

    mon.mGetHdr(add_dir+'/'+myfilelist[1],"master.hdr")
    print('getting fits header')
    mon.mImgtbl(add_dir,"ima.tbl")
    print('creating image table for images to stack')
    mon.mProjExec(add_dir,"ima.tbl","master.hdr",projdir="montage_rot",debug=1)
    print('reprojecting images')
    mon.mImgtbl("montage_rot","images.tbl")
    print('creating image table for reprojected images')
    mon.mOverlaps("images.tbl","diffs.tbl")
    print('determining overlaps')
    mon.mDiffExec("montage_rot","diffs.tbl","master.hdr","montage_diff")

    mon.mFitExec("diffs.tbl", "fits.tbl", "montage_diff")
    print('fitting overlaps')
    mon.mBgModel("images.tbl", "fits.tbl", "corrections.tbl")
    print('determining background model')
    mon.mBgExec("montage_rot", "images.tbl", "corrections.tbl", "montage_corr")
    print('fitting background model')
    mon.mAdd("montage_corr", "images.tbl", "master.hdr" ,"test.fits",haveAreas=True,coadd=1)
    print('stacking images')

    os.system('rm -rf montage_rot')
    os.system('rm -rf montage_diff')
    os.system('rm -rf montage_corr')
    os.system('rm ima.tbl images.tbl diffs.tbl fits.tbl corrections.tbl  master.hdr')
    print('removed temp files')

    sum_exptime = 0.0
    sum_mjd = 0.0
    for ffff in myfilelist:
            data, header = pyfits.getdata(add_dir+'/'+ffff, header=True)
            sum_exptime = sum_exptime + header['EXPTIME']
            sum_mjd = sum_mjd + header['MJD-OBS']
    #calculate the total exposure time and average mjd
    exptime = sum_exptime
    mjd = sum_mjd / len(myfilelist)
    #read in the stacked file and change the header entries
    data, header = pyfits.getdata('test.fits', header=True)
    header['EXPTIME'] = exptime
    header['MJD-OBS'] = mjd
    #write out the changed file
    outfilename =  pathout+'/'+'lco_'+objectname+'_'+fil+'_'+username+'.fits'
    pyfits.writeto(outfilename, data, header, overwrite=True)
    #clean up the directory
    os.system("rm test.fits *_area.fits")
    print('***finished stacking images***')

In [ ]:
fil_select=widget.Dropdown(
    options=list(np.unique(filtlist)),
    description='Select Filter:'
)
display(fil_select)

In [ ]:
stack_images(filters,fil_select.value,pathout,username,objectname)

In [ ]:
for image in os.listdir(pathout):
    if image.endswith('fits'):
        file=pathout+'/'+image
        mon.mViewer("-ct 1 -gray "+file+" -2s max gaussian-log -out test.png", "", mode=2)
        print('now showing ',image)
        display(Image('test.png'))
        os.system('rm test.png')